In [ ]:
from music21 import *
import pandas as pd
import os

In [ ]:
#!cd /content/drive
!cmd

/bin/bash: line 1: cmd: command not found


In [47]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


In [44]:
# Replace 'file_path.csv' with the path to your CSV file
file_path = '/content/drive/MyDrive/scores_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)
df = df.drop(columns=['Work', 'Source', 'MBID'])
# Display the DataFrame
df.head(5)

,File name,Role type,Shengqiang,Banshi,Accompaniment
0,daeh-CanQiQi-WuLongZuo,dan,erhuang,manban,yes
1,daeh-LiYanFei-DaBaoGuo,dan,erhuang,manban,no
2,daeh-LiYanFei-ErJinGong,dan,erhuang,zhongsanyan,no
3,daeh-NaRiFeng-YuanXiaoMi,dan,erhuang,yuanban,no
4,daeh-NuZiYou-TouJunBieYao,dan,erhuang,manban,no


In [56]:
len(scores)

108

In [52]:
def find_xml_files(folder_path):
    xml_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.xml') or file.endswith('.musicxml'):
                xml_files.append(os.path.join(root, file))
    return xml_files

folder_path = '/content/drive/MyDrive/JingjuMusicScoresCollection-v3'
scores = find_xml_files(folder_path)
df['File Path'] = scores
pd.set_option('display.max_colwidth', None)
df.head(5)

,File name,Role type,Shengqiang,Banshi,Accompaniment,File Path
0,daeh-CanQiQi-WuLongZuo,dan,erhuang,manban,yes,/content/drive/MyDrive/JingjuMusicScoresCollection-v3/Jingju Scores Dataset/MusicXML/lsxp-WoBenShi-KongChengJi-1.xml
1,daeh-LiYanFei-DaBaoGuo,dan,erhuang,manban,no,/content/drive/MyDrive/JingjuMusicScoresCollection-v3/Jingju Scores Dataset/MusicXML/lseh-YiLunMing-ZhuoFangCao-1.xml
2,daeh-LiYanFei-ErJinGong,dan,erhuang,zhongsanyan,no,/content/drive/MyDrive/JingjuMusicScoresCollection-v3/Jingju Scores Dataset/MusicXML/lseh-YiLunMing-ZhuoFangSuDian.xml
3,daeh-NaRiFeng-YuanXiaoMi,dan,erhuang,yuanban,no,/content/drive/MyDrive/JingjuMusicScoresCollection-v3/Jingju Scores Dataset/MusicXML/lsxp-ZiCongQian-ZhuLianZhai.xml
4,daeh-NuZiYou-TouJunBieYao,dan,erhuang,manban,no,/content/drive/MyDrive/JingjuMusicScoresCollection-v3/Jingju Scores Dataset/MusicXML/lsxp-WeiCengKai-QiYuanBao.xml


In [58]:
def get_melodic_patterns(score, min_notes, max_notes):
    melodic_patterns = {}
    all_notes = []

    for part in score.parts:
        all_notes.extend(part.recurse().notes)

    for note_length in range(min_notes, max_notes + 1):
        patterns = {}
        for i in range(len(all_notes) - note_length + 1):
            pattern = tuple(n.pitch for n in all_notes[i:i+note_length])
            if pattern in patterns:
                patterns[pattern] += 1
            else:
                patterns[pattern] = 1
        melodic_patterns[note_length] = patterns

    return melodic_patterns

def find_most_repeated_patterns(melodic_patterns):
    most_repeated_patterns = {}
    for pattern_length, patterns in melodic_patterns.items():
        most_repeated_patterns[pattern_length] = max(patterns.items(), key=lambda x: x[1])
    return most_repeated_patterns

# Replace 'your_score.xml' with the path to your MusicXML file
score = converter.parse('/content/drive/MyDrive/JingjuMusicScoresCollection-v3/Jingju Scores Dataset/MusicXML/daeh-LiYanFei-ErJinGong.xml')

min_notes = 3  # Minimum number of notes in a pattern
max_notes = 8  # Maximum number of notes in a pattern

melodic_patterns = get_melodic_patterns(score, min_notes, max_notes)
most_repeated_patterns = find_most_repeated_patterns(melodic_patterns)





In [59]:
min_notes = 3  # Minimum number of notes in a pattern
max_notes = 8  # Maximum number of notes in a pattern

for file_index, file_path in enumerate(df['File Path']):
    score = converter.parse(file_path)
    melodic_patterns = get_melodic_patterns(score, min_notes, max_notes)
    most_repeated_patterns = find_most_repeated_patterns(melodic_patterns)

    for note_length, (pattern, count) in most_repeated_patterns.items():
        pattern_str = ", ".join(str(p) for p in pattern)
        pattern_column = f'{note_length} note pattern'
        repetition_column = f'{note_length} note repetition'
        df.loc[file_index, pattern_column] = pattern_str
        df.loc[file_index, repetition_column] = count

In [ ]:
df.head(5)

,File name,Role type,Shengqiang,Banshi,Accompaniment,File Path,3 note pattern,3 note repetition,4 note pattern,4 note repetition,5 note pattern,5 note repetition,6 note pattern,6 note repetition,7 note pattern,7 note repetition,8 note pattern,8 note repetition
0,daeh-CanQiQi-WuLongZuo,dan,erhuang,manban,yes,JingjuMusicScoresCollection-v3\Jingju Scores Dataset\MusicXML\daeh-CanQiQi-WuLongZuo.xml,"C#4, B3, C#4",46,"D#4, C#4, B3, C#4",24,"D#4, C#4, B3, C#4, E4",9,"F#4, G#4, D#4, C#4, B3, C#4",7,"C#4, C#4, C#4, C#4, C#4, C#4, C#4",4,"D#4, C#4, B3, C#4, E4, C#4, E4, F#4",3
1,daeh-LiYanFei-DaBaoGuo,dan,erhuang,manban,no,JingjuMusicScoresCollection-v3\Jingju Scores Dataset\MusicXML\daeh-LiYanFei-DaBaoGuo.xml,"B4, G#4, B4",32,"B4, G#4, B4, C#5",23,"B4, B4, G#4, B4, C#5",16,"G#4, B4, B4, G#4, B4, C#5",9,"C#5, C#5, G#4, B4, B4, G#4, B4",8,"C#5, C#5, G#4, B4, B4, G#4, B4, C#5",8
2,daeh-LiYanFei-ErJinGong,dan,erhuang,zhongsanyan,no,JingjuMusicScoresCollection-v3\Jingju Scores Dataset\MusicXML\daeh-LiYanFei-ErJinGong.xml,"D#4, C#4, B3",23,"B4, G#4, B4, C#5",15,"B4, B4, G#4, B4, C#5",8,"C#5, B4, G#4, B4, C#5, B4",5,"C#5, B4, G#4, B4, C#5, B4, E5",4,"B4, C#5, B4, G#4, B4, C#5, B4, E5",3
3,daeh-NaRiFeng-YuanXiaoMi,dan,erhuang,yuanban,no,JingjuMusicScoresCollection-v3\Jingju Scores Dataset\MusicXML\daeh-NaRiFeng-YuanXiaoMi.xml,"C#4, B3, C#4",28,"C#4, B3, C#4, E4",17,"B4, C#5, E5, B4, C#5",7,"F#4, G#4, F#4, E4, C#4, B3",6,"F#4, G#4, F#4, E4, C#4, B3, C#4",6,"F#4, E4, C#4, B3, C#4, B3, C#4, E4",6
4,daeh-NuZiYou-TouJunBieYao,dan,erhuang,manban,no,JingjuMusicScoresCollection-v3\Jingju Scores Dataset\MusicXML\daeh-NuZiYou-TouJunBieYao.xml,"D#5, C#5, B4",16,"B4, G#4, B4, C#5",11,"B4, G#4, B4, C#5, B4",5,"D#5, F#5, D#5, C#5, B4, C#5",4,"C#5, D#5, F#5, D#5, C#5, B4, C#5",3,"F#5, D#5, C#5, B4, B4, G#4, B4, C#5",2
